In [31]:
import torch
from models.res_unet_adrian import UNet as unet

In [32]:
batch_size = 4
batch = torch.zeros([batch_size, 3, 80, 80], dtype=torch.float32)

In [33]:
batch.shape

torch.Size([4, 3, 80, 80])

N_classes = 2

In [34]:
n_classes = 2

In [35]:
model = unet(in_c=3, n_classes=n_classes, layers=[8,16], conv_bridge=True, shortcut=True)

In [36]:
logits = model(batch)
logits.shape, logits.dtype

(torch.Size([4, 2, 80, 80]), torch.float32)

In [37]:
labels = torch.randn([batch_size, 80,80]).ge(0).float()
labels.shape, labels.dtype

(torch.Size([4, 80, 80]), torch.float32)

In [38]:
criterion = torch.nn.CrossEntropyLoss()

In [39]:
# loss = criterion(logits, labels.unsqueeze(dim=1).float())  # BCEWithLogitsLoss()
loss = criterion(logits, labels.long())  

In [40]:
loss

tensor(0.7731, grad_fn=<NllLoss2DBackward>)

### dice loss

In [41]:
import torch.nn.functional as F
input_soft = F.softmax(logits, dim=1)

In [42]:
def one_hot(labels, n_classes, eps=1e-6):
    shape = labels.shape
    one_hot = torch.zeros(shape[0], n_classes, *shape[1:])
    return one_hot.scatter_(1, labels.unsqueeze(1), 1.0) + eps

In [43]:
labels.shape

torch.Size([4, 80, 80])

In [44]:
labels_one_hot = one_hot(labels.long(), n_classes=2)
labels_one_hot.shape

torch.Size([4, 2, 80, 80])

In [46]:
# compute the actual dice score
dims = (1, 2, 3)
intersection = torch.sum(input_soft * labels_one_hot, dims)
cardinality = torch.sum(input_soft + labels_one_hot, dims)

dice_score = 2. * intersection / (cardinality + 1e-6)
print(torch.mean(torch.tensor(1.) - dice_score))

tensor(0.4987, grad_fn=<MeanBackward0>)


In [47]:
intersection.shape, cardinality.shape

(torch.Size([4]), torch.Size([4]))

# N_classes=1

In [16]:
n_classes = 1

In [17]:
model = unet(in_c=3, n_classes=n_classes, layers=[8,16], conv_bridge=True, shortcut=True)

In [18]:
logits = model(batch)
logits.shape, logits.dtype

(torch.Size([4, 1, 80, 80]), torch.float32)

In [19]:
labels = torch.randn([batch_size, 80,80]).ge(0).float()
labels.shape, labels.dtype

(torch.Size([4, 80, 80]), torch.float32)

In [20]:
criterion = torch.nn.BCEWithLogitsLoss()

In [21]:
loss = criterion(logits, labels.unsqueeze(dim=1).float())  # BCEWithLogitsLoss()

In [22]:
loss

tensor(0.7649, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

### dice loss

In [23]:
import torch.nn.functional as F
input_soft = torch.sigmoid(logits)
input_soft.shape

torch.Size([4, 1, 80, 80])

In [24]:
def one_hot(labels, n_classes, eps=1e-6):
    shape = labels.shape
    one_hot = torch.zeros(shape[0], n_classes, *shape[1:])
    return one_hot.scatter_(1, labels.unsqueeze(1), 1.0) + eps

In [48]:
labels.ndim

3

In [25]:
labels.shape

torch.Size([4, 80, 80])

In [28]:
dims = (1, 2, 3)
intersection = torch.sum(input_soft * labels.unsqueeze(dim=1), dims)
cardinality = torch.sum(input_soft + labels.unsqueeze(dim=1), dims)
dice_score = 2. * intersection / (cardinality + 1e-6)
print(torch.mean(torch.tensor(1.) - dice_score))

tensor(0.4619, grad_fn=<MeanBackward0>)


In [30]:
intersection.shape, cardinality.shape

(torch.Size([4]), torch.Size([4]))

In [ ]:
# compute the actual dice score
dims = (1, 2, 3)
intersection = torch.sum(input_soft * labels_one_hot, dims)
cardinality = torch.sum(input_soft + labels_one_hot, dims)

dice_score = 2. * intersection / (cardinality + 1e-6)
print(torch.mean(torch.tensor(1.) - dice_score))

In [ ]:
import argparse
from PIL import Image
import os, sys
import os.path as osp
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score
from utils.evaluation import dice_score

In [ ]:
def get_labels_preds(path_to_preds, csv_path):
    df = pd.read_csv(csv_path)
    im_list, mask_list, gt_list = df.im_paths, df.mask_paths, df.gt_paths

    all_bin_preds = []
    all_preds = []
    all_gts = []
    for i in range(len(gt_list)):
        im_path = im_list[i].rsplit('/', 1)[-1]
        pred_path = osp.join(path_to_preds, im_path[:-4] + '.png')
        bin_pred_path = osp.join(path_to_preds, im_path[:-4] + '_binary.png')
        gt_path = gt_list[i]
        mask_path = mask_list[i]

        gt = np.array(Image.open(gt_path)).astype(bool)
        mask = np.array(Image.open(mask_path).convert('L')).astype(bool)
        from skimage import img_as_float

        try: pred = img_as_float(np.array(Image.open(pred_path)))
        except FileNotFoundError:
            sys.exit('---- no predictions found (maybe run first generate_results.py?) ---- ')
        # os.remove(pred_path)
        bin_pred = np.array(Image.open(bin_pred_path).convert('L')).astype(bool)
        gt_flat = gt.ravel()
        mask_flat = mask.ravel()
        pred_flat = pred.ravel()
        bin_pred_flat = bin_pred.ravel()
        # do not consider pixels out of the FOV
        noFOV_gt = gt_flat[mask_flat == True]
        noFOV_pred = pred_flat[mask_flat == True]
        noFOV_bin_pred = bin_pred_flat[mask_flat == True]

        # accumulate gt pixels and prediction pixels
        all_preds.append(noFOV_pred)
        all_bin_preds.append(noFOV_bin_pred)
        all_gts.append(noFOV_gt)

    return np.hstack(all_preds), np.hstack(all_bin_preds), np.hstack(all_gts)

In [ ]:
def cutoff_youden(fpr, tpr, thresholds):
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    return optimal_threshold


def cutoff_dice(preds, gts):
    dice_scores = []
    thresholds = np.linspace(0, 1, 256)
    for i in tqdm(range(len(thresholds))):
        thresh = thresholds[i]
        hard_preds = preds>thresh
        dice_scores.append(dice_score(gts, hard_preds))
    dices = np.array(dice_scores)
    optimal_threshold = thresholds[dices.argmax()]
    return optimal_threshold

def cutoff_accuracy(preds, gts):
    accuracy_scores = []
    thresholds = np.linspace(0, 1, 256)
    for i in tqdm(range(len(thresholds))):
        thresh = thresholds[i]
        hard_preds = preds > thresh
        accuracy_scores.append(accuracy_score(gts.astype(np.bool), hard_preds.astype(np.bool)))
    accuracies = np.array(accuracy_scores)
    optimal_threshold = thresholds[accuracies.argmax()]
    return optimal_threshold    

In [ ]:
def compute_performance(preds, bin_preds, gts, save_path=None, opt_threshold=None,
                        cut_off='dice', mode='train', no_auc=False):
    if no_auc: global_auc=0
    else:
        fpr, tpr, thresholds = roc_curve(gts, preds)
        global_auc = auc(fpr, tpr)

    if save_path is not None and no_auc==False:
        fig = plt.figure(figsize=(8, 8))
        plt.plot(fpr, tpr, label='ROC curve')
        ll = 'AUC = {:4f}'.format(global_auc)
        plt.legend([ll], loc='lower right')
        fig.tight_layout()
        if opt_threshold is None:
            if mode=='train':
                plt.savefig(osp.join(save_path,'ROC_train.png'))
            elif mode=='val':
                plt.savefig(osp.join(save_path, 'ROC_val.png'))
        else:
            plt.savefig(osp.join(save_path, 'ROC_test.png'))

    if opt_threshold is None:
        if cut_off == 'acc':
            # this would be to get accuracy-maximizing threshold
            opt_threshold = cutoff_accuracy(preds, gts)
        elif cut_off == 'dice':
            # this would be to get dice-maximizing threshold
            opt_threshold = cutoff_dice(preds, gts)
        else:
            opt_threshold = cutoff_youden(fpr, tpr, thresholds)
    if no_auc: print('Computing Accuracy...')
    acc = accuracy_score(gts, preds > opt_threshold)
    if no_auc: print('Accuracy computed')
    dice = dice_score(gts, preds > opt_threshold)
    if no_auc: print('Dice from ROC-based threshold computed')
    dice_from_bin = dice_score(gts, bin_preds)
    if no_auc: print('Dice from threshold-free binarized computed')

    if no_auc:
        print('Can\'t compute conf_mat on full HRF dataset at once')
        specificity, sensitivity = 0,0
    else:
        tn, fp, fn, tp = confusion_matrix(gts, preds > opt_threshold).ravel()
        specificity = tn / (tn + fp)
        sensitivity = tp / (tp + fn)

    return global_auc, acc, dice, dice_from_bin, specificity, sensitivity, opt_threshold

In [ ]:
experiment_path = 'experiments/best_DRIVE/'
results_path = 'results/'
train_dataset = 'DRIVE'
test_dataset = 'AUCKLAND_0'
if experiment_path is None: raise Exception('must specify experiment_path, sorry')
exp_name = experiment_path
cut_off = 'dice'

In [ ]:
print('* Analyzing performance in ' + train_dataset + ' training set -- Obtaining optimal threshold')
path_to_preds = osp.join(results_path, train_dataset, exp_name)
save_path = osp.join(path_to_preds, 'perf')

perf_csv_path = osp.join(save_path, 'training_performance.csv')
if osp.exists(perf_csv_path):
    global_auc_tr, acc_tr, dice_tr, dice_from_bin_tr,\
    spec_tr, sens_tr, opt_thresh_tr = pd.read_csv(perf_csv_path).values[0]
    print('-- Performance in ' + train_dataset + ' training set had been pre-computed, '
                                                 'optimal threshold = {:.4f}'.format(opt_thresh_tr))
else:
    csv_path = osp.join('data', train_dataset, 'train.csv')
    if train_dataset == 'HRF': csv_path = osp.join('data', train_dataset, 'train_fullRes.csv')
    preds, bin_preds, gts = get_labels_preds(path_to_preds, csv_path = csv_path)
    os.makedirs(save_path, exist_ok=True)
    global_auc_tr, acc_tr, dice_tr, dice_from_bin_tr,\
    spec_tr, sens_tr, opt_thresh_tr = compute_performance(preds, bin_preds, gts, save_path=save_path,
                                                          opt_threshold=None, cut_off=cut_off, mode='train')
    perf_df_train = pd.DataFrame({'auc': global_auc_tr,
                                'acc': acc_tr,
                                'dice/F1': dice_tr,
                                'dice/F1_from_bin': dice_from_bin_tr,
                                'spec': spec_tr,
                                'sens': sens_tr,
                                'opt_t': opt_thresh_tr}, index=[0])
    perf_df_train.to_csv(perf_csv_path, index=False)

In [ ]:
print('* Analyzing performance in ' + train_dataset + ' validation set')
perf_csv_path = osp.join(save_path, 'validation_performance.csv')
if osp.exists(perf_csv_path):
    global_auc_vl, acc_vl, dice_vl, dice_from_bin_vl,\
    spec_vl, sens_vl = pd.read_csv(perf_csv_path).values[0]
    print('-- Performance in ' + train_dataset + ' validation set had been pre-computed')
else:
    csv_path = osp.join('data', train_dataset, 'val.csv')
    if train_dataset=='HRF': csv_path = osp.join('data', train_dataset, 'val_fullRes.csv')
    preds, bin_preds, gts = get_labels_preds(path_to_preds, csv_path = csv_path)
    global_auc_vl, acc_vl, dice_vl, dice_from_bin_vl,\
    spec_vl, sens_vl, _ = compute_performance(preds, bin_preds, gts, save_path=save_path,
                                                      opt_threshold=opt_thresh_tr, cut_off=cut_off, mode='train')
    perf_df_train = pd.DataFrame({'auc': global_auc_vl,
                                  'acc': acc_vl,
                                  'dice/F1': dice_vl,
                                  'dice/F1_from_bin': dice_from_bin_vl,
                                  'spec': spec_vl,
                                  'sens': sens_vl}, index=[0])
    perf_df_train.to_csv(perf_csv_path, index=False)

In [ ]:
print('* Analyzing performance in ' + test_dataset + ' test set')
path_to_preds = osp.join(results_path, test_dataset, exp_name)
save_path = osp.join(path_to_preds, 'perf')
os.makedirs(save_path, exist_ok=True)
perf_csv_path = osp.join(save_path, 'test_performance.csv')

In [ ]:
path_test_csv = osp.join('data', test_dataset, 'test_all.csv')

In [ ]:
def get_labels_preds(path_to_preds, csv_path):
    df = pd.read_csv(csv_path)
    im_list, mask_list, gt_list = df.im_paths, df.mask_paths, df.gt_paths

    all_bin_preds = []
    all_preds = []
    all_gts = []
    for i in range(2,len(gt_list)):
        im_path = im_list[i].rsplit('/', 1)[-1]
        pred_path = osp.join(path_to_preds, im_path[:-4] + '.png')
        bin_pred_path = osp.join(path_to_preds, im_path[:-4] + '_binary.png')
        gt_path = gt_list[i]
        mask_path = mask_list[i]
        
        gt = np.array(Image.open(gt_path)).astype(bool)
        mask = np.array(Image.open(mask_path).convert('L')).astype(bool)
        from skimage import img_as_float

        try: pred = img_as_float(np.array(Image.open(pred_path)))
        except FileNotFoundError:
            sys.exit('---- no predictions found (maybe run first generate_results.py?) ---- ')
        # os.remove(pred_path)
        
        bin_pred = np.array(Image.open(bin_pred_path).convert('L')).astype(bool)
        return pred, bin_pred, gt, mask
        gt_flat = gt.ravel()
        mask_flat = mask.ravel()
        pred_flat = pred.ravel()
        bin_pred_flat = bin_pred.ravel()
        # do not consider pixels out of the FOV
        noFOV_gt = gt_flat[mask_flat == True]
        noFOV_pred = pred_flat[mask_flat == True]
        noFOV_bin_pred = bin_pred_flat[mask_flat == True]

        # accumulate gt pixels and prediction pixels
        all_preds.append(noFOV_pred)
        all_bin_preds.append(noFOV_bin_pred)
        all_gts.append(noFOV_gt)

    return np.hstack(all_preds), np.hstack(all_bin_preds), np.hstack(all_gts)

In [ ]:
pred, bin_pred, gt, mask = get_labels_preds(path_to_preds, csv_path = path_test_csv)

In [ ]:
import matplotlib.pyplot as plt

def imshow_pair(im, gdt):
    f, ax = plt.subplots(1, 2, figsize=(12,6))
    np_im = np.asarray(im)
    np_gdt = np.asarray(gdt)
    if len(np_im.shape) == 2:
        ax[0].imshow(np_im, cmap='gray'),  ax[0].axis('off')
    else:
        ax[0].imshow(np_im),  ax[0].axis('off')
    if len(np_gdt.shape) == 2:
        ax[1].imshow(np.asarray(gdt), cmap = 'gray'), ax[1].axis('off')
    else:
        ax[1].imshow(np.asarray(gdt)), ax[1].axis('off')
    plt.tight_layout()

In [ ]:
imshow_pair(pred, mask)

In [ ]:
imshow_pair(bin_pred, gt)

In [ ]:
gt_flat = gt.ravel()
mask_flat = mask.ravel()
pred_flat = pred.ravel()
bin_pred_flat = bin_pred.ravel()
# do not consider pixels out of the FOV
noFOV_gt = gt_flat[mask_flat == True]
noFOV_pred = pred_flat[mask_flat == True]
noFOV_bin_pred = bin_pred_flat[mask_flat == True]

In [ ]:
from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score
from utils.evaluation import dice_score

In [ ]:
dice_score(noFOV_gt, noFOV_bin_pred)

In [ ]:
np.sum(noFOV_bin_pred[noFOV_gt==1])*2.0 / (np.sum(noFOV_bin_pred) + np.sum(noFOV_gt))

In [ ]:
np.unique(gt)

In [ ]:
np.unique(bin_pred)

In [ ]:
ims

In [ ]:
preds, bin_preds, gts = get_labels_preds(path_to_preds, csv_path = path_test_csv)

In [ ]:
global_auc_test, acc_test, dice_test, dice_from_bin_test, spec_test, sens_test, _ = \
    compute_performance(preds, bin_preds, gts, save_path=save_path, opt_threshold=opt_thresh_tr, no_auc=no_auc)
perf_df_test = pd.DataFrame({'auc': global_auc_test,
                             'acc': acc_test,
                             'dice/F1': dice_test,
                             'dice/F1_from_bin':dice_from_bin_test,
                             'spec': spec_test,
                             'sens': sens_test}, index=[0])
perf_df_test.to_csv(perf_csv_path, index=False)
print('* Done')
print('AUC in Train/Val/Test set is {:.4f}/{:.4f}/{:.4f}'.format(global_auc_tr, global_auc_vl, global_auc_test))
print('Accuracy in Train/Val/Test set is {:.4f}/{:.4f}/{:.4f}'.format(acc_tr, acc_vl, acc_test))
print('Dice/F1 score in Train/Val/Test set is {:.4f}/{:.4f}/{:.4f}'.format(dice_tr, dice_vl, dice_test))
print('Dice/F1 score **from binarized** in Train/Val/Test set is {:.4f}/{:.4f}/{:.4f}'.format(
       dice_from_bin_tr, dice_from_bin_vl, dice_from_bin_test))
print('Specificity in Train/Val/Test set is {:.4f}/{:.4f}/{:.4f}'.format(spec_tr, spec_vl, spec_test))
print('Sensitivity in Train/Val/Test set is {:.4f}/{:.4f}/{:.4f}'.format(sens_tr, sens_vl, sens_test))
print('ROC curve plots saved to ', save_path)

In [ ]:
import argparse
from PIL import Image
import os, sys
import os.path as osp
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score
from utils.evaluation import dice_score

%load_ext autoreload
%autoreload 2

In [ ]:
def get_labels_preds(path_to_preds, csv_path):
    df = pd.read_csv(csv_path)
    im_list, mask_list, gt_list = df.im_paths, df.mask_paths, df.gt_paths

    all_bin_preds = []
    all_preds = []
    all_gts = []
    for i in range(len(gt_list)):
        im_path = im_list[i].rsplit('/', 1)[-1]
        pred_path = osp.join(path_to_preds, im_path[:-4] + '.png')
        bin_pred_path = osp.join(path_to_preds, im_path[:-4] + '_binary.png')
        gt_path = gt_list[i]
        mask_path = mask_list[i]

        gt = np.array(Image.open(gt_path)).astype(bool)
        mask = np.array(Image.open(mask_path).convert('L')).astype(bool)
        from skimage import img_as_float
        try: pred = img_as_float(np.array(Image.open(pred_path)))
        except FileNotFoundError:
            sys.exit('---- no predictions found (maybe run first generate_results.py?) ---- ')

        bin_pred = np.array(Image.open(bin_pred_path).convert('L')).astype(bool)
        gt_flat = gt.ravel()
        mask_flat = mask.ravel()
        pred_flat = pred.ravel()
        bin_pred_flat = bin_pred.ravel()
        # do not consider pixels out of the FOV
        noFOV_gt = gt_flat[mask_flat == True]
        noFOV_pred = pred_flat[mask_flat == True]
        noFOV_bin_pred = bin_pred_flat[mask_flat == True]

        # accumulate gt pixels and prediction pixels
        all_preds.append(noFOV_pred)
        all_bin_preds.append(noFOV_bin_pred)
        all_gts.append(noFOV_gt)

    return np.hstack(all_preds), np.hstack(all_bin_preds), np.hstack(all_gts)

In [ ]:
method = 'deep_vessels'
dataset = 'DRIVE'

print('* Analyzing performance of '+ method +' in ' + dataset + ' test set')
path_to_preds = osp.join('results', dataset, 'experiments', method)
csv_name = 'test.csv'

path_test_csv = osp.join('data', dataset, csv_name)

In [ ]:
preds, bin_preds, gts = get_labels_preds(path_to_preds, csv_path = path_test_csv)

In [ ]:
accuracy_score(bin_preds, gts)

In [ ]:
global_auc_test, acc_test, dice_test, spec_test, sens_test =  compute_performance(preds, bin_preds, gts)
print('* Done')
print('AUC in Test set is {:.4f}'.format(global_auc_test))
print('Accuracy in Test set is {:.4f}'.format(acc_test))
print('Dice/F1 score in Test set is {:.4f}'.format(dice_test))
print('Specificity in Test set is {:.4f}'.format(spec_test))
print('Sensitivity in Test set is {:.4f}'.format(sens_test))

In [ ]:
config_file = 'experiments/2020-01-15-10:2735/config.cfg'

In [ ]:
results_path = 'results/'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = 'DRIVE'
binarize = 'otsu'
tta = 'from_preds'

In [ ]:
if config_file is not None:
    if not osp.isfile(config_file): raise Exception('non-existent config file')
    with open(config_file, 'r') as f:
        args = json.load(f)

In [ ]:
args

In [ ]:
layers = args['layers']
layers = args['layers'].split('/')
layers = list(map(int, layers))
n_classes = args['n_classes']
conv_bridge = args['conv_bridge']
shortcut = args['shortcut']
experiment_path = args['experiment_path']

In [ ]:
if experiment_path is None: raise Exception('must specify path to experiment')

In [ ]:
model= unet(in_c=3, n_classes=n_classes, layers=layers, conv_bridge=conv_bridge, shortcut=shortcut).to(device)

In [ ]:
path = osp.join(experiment_path, 'model_checkpoint.pth')
checkpoint = torch.load(path, map_location=torch.device('cpu'))

In [ ]:
def load_model(model, experiment_path, device):
    checkpoint_path = osp.join(experiment_path, 'model_checkpoint.pth')
    # checkpoint = torch.load(checkpoint_path)
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    return model, checkpoint['stats']

In [ ]:
model, stats = load_model(model, experiment_path, device)

In [ ]:
from PIL import Image
import os, sys
import os.path as osp
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score
from utils.evaluation import dice_score

In [ ]:
def get_labels_preds(path_to_preds, csv_path):
    df = pd.read_csv(csv_path)
    im_list, mask_list, gt_list = df.im_paths, df.mask_paths, df.gt_paths

    all_bin_preds = []
    all_preds = []
    all_gts = []
    for i in range(len(gt_list)):
        im_path = im_list[i].rsplit('/', 1)[-1]
        pred_path = osp.join(path_to_preds, im_path[:-4] + '.png')
        bin_pred_path = osp.join(path_to_preds, im_path[:-4] + '_binary.png')
        gt_path = gt_list[i]
        mask_path = mask_list[i]

        gt = np.array(Image.open(gt_path)).astype(bool)
        mask = np.array(Image.open(mask_path).convert('L')).astype(bool)
        from skimage import img_as_float

        try: pred = img_as_float(np.array(Image.open(pred_path)))
        except FileNotFoundError:
            sys.exit('---- no predictions found (maybe run first generate_results.py?) ---- ')
        # os.remove(pred_path)
        bin_pred = np.array(Image.open(bin_pred_path).convert('L')).astype(bool)
        gt_flat = gt.ravel()
        mask_flat = mask.ravel()
        pred_flat = pred.ravel()
        bin_pred_flat = bin_pred.ravel()
        # do not consider pixels out of the FOV
        noFOV_gt = gt_flat[mask_flat == True]
        noFOV_pred = pred_flat[mask_flat == True]
        noFOV_bin_pred = bin_pred_flat[mask_flat == True]

        # accumulate gt pixels and prediction pixels
        all_preds.append(noFOV_pred)
        all_bin_preds.append(noFOV_bin_pred)
        all_gts.append(noFOV_gt)

    return np.hstack(all_preds), np.hstack(all_bin_preds), np.hstack(all_gts)

In [ ]:
def cutoff_youden(fpr, tpr, thresholds):
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    return optimal_threshold


def cutoff_dice(preds, gts):
    dice_scores = []
    thresholds = np.linspace(0, 1, 256)
    for i in tqdm(range(len(thresholds))):
        thresh = thresholds[i]
        hard_preds = preds>thresh
        dice_scores.append(dice_score(gts, hard_preds))
    dices = np.array(dice_scores)
    optimal_threshold = thresholds[dices.argmax()]
    return optimal_threshold

def cutoff_accuracy(preds, gts):
    accuracy_scores = []
    thresholds = np.linspace(0, 1, 256)
    for i in tqdm(range(len(thresholds))):
        thresh = thresholds[i]
        hard_preds = preds > thresh
        accuracy_scores.append(accuracy_score(gts.astype(np.bool), hard_preds.astype(np.bool)))
    accuracies = np.array(accuracy_scores)
    optimal_threshold = thresholds[accuracies.argmax()]
    return optimal_threshold

In [ ]:
def compute_performance(preds, bin_preds, gts, save_path=None, opt_threshold=None,
                        cut_off='dice', mode='train', no_auc=False):
    if no_auc: global_auc=0
    else:
        fpr, tpr, thresholds = roc_curve(gts, preds)
        global_auc = auc(fpr, tpr)

    if save_path is not None and no_auc==False:
        fig = plt.figure(figsize=(8, 8))
        plt.plot(fpr, tpr, label='ROC curve')
        ll = 'AUC = {:4f}'.format(global_auc)
        plt.legend([ll], loc='lower right')
        fig.tight_layout()
        if opt_threshold is None:
            if mode=='train':
                plt.savefig(osp.join(save_path,'ROC_train.png'))
            elif mode=='val':
                plt.savefig(osp.join(save_path, 'ROC_val.png'))
        else:
            plt.savefig(osp.join(save_path, 'ROC_test.png'))

    if opt_threshold is None:
        if cut_off == 'acc':
            # this would be to get accuracy-maximizing threshold
            opt_threshold = cutoff_accuracy(preds, gts)
        elif cut_off == 'dice':
            # this would be to get dice-maximizing threshold
            opt_threshold = cutoff_dice(preds, gts)
        else:
            opt_threshold = cutoff_youden(fpr, tpr, thresholds)
    if no_auc: print('Computing Accuracy...')
    acc = accuracy_score(gts, preds > opt_threshold)
    if no_auc: print('Accuracy computed')
    dice = dice_score(gts, preds > opt_threshold)
    if no_auc: print('Dice from ROC-based threshold computed')
    dice_from_bin = dice_score(gts, bin_preds)
    if no_auc: print('Dice from threshold-free binarized computed')

    if no_auc:
        print('Can\'t compute conf_mat on full HRF dataset at once')
        specificity, sensitivity = 0,0
    else:
        tn, fp, fn, tp = confusion_matrix(gts, preds > opt_threshold).ravel()
        specificity = tn / (tn + fp)
        sensitivity = tp / (tp + fn)

    return global_auc, acc, dice, dice_from_bin, specificity, sensitivity, opt_threshold

In [ ]:
exp_path = 'experiments/'
results_path = 'results/'

train_dataset = 'HRF'
test_dataset = 'HRF'
exp_name = 'experiments/2020-01-14-18:2721/'
cut_off = 'dice'

In [ ]:
from utils.evaluation import fast_auc as auc

In [ ]:
path_to_preds = osp.join(results_path, train_dataset, exp_name)
save_path = osp.join(path_to_preds, 'perf')

perf_csv_path = osp.join(save_path, 'validation_performance.csv')
if osp.exists(perf_csv_path):
    global_auc_vl, acc_vl, dice_vl, dice_from_bin_vl,\
    spec_vl, sens_vl, opt_thresh_vl = pd.read_csv(perf_csv_path).values[0]
    print('-- Performance in validation set had been pre-computed, optimal threshold = {:.4f}'.format(
        opt_thresh_vl))
else:
    csv_path = osp.join('data', train_dataset, 'val.csv')
    if train_dataset == 'HRF': csv_path = osp.join('data', train_dataset, 'val_fullRes.csv')
    preds, bin_preds, gts = get_labels_preds(path_to_preds, csv_path = csv_path)
    os.makedirs(save_path, exist_ok=True)
    global_auc_vl, acc_vl, dice_vl, dice_from_bin_vl,\
    spec_vl, sens_vl, opt_thresh_vl = compute_performance(preds, bin_preds, gts, save_path=save_path,
                                                          opt_threshold=None, cut_off=cut_off, mode='val')
    perf_df_val = pd.DataFrame({'auc': global_auc_vl,
                                'acc': acc_vl,
                                'dice/F1': dice_vl,
                                'dice/F1_from_bin': dice_from_bin_vl,
                                'spec': spec_vl,
                                'sens': sens_vl,
                                'opt_t': opt_thresh_vl}, index=[0])
    perf_df_val.to_csv(perf_csv_path, index=False)

In [ ]:
print('* Analyzing performance in training set')
perf_csv_path = osp.join(save_path, 'training_performance.csv')
if osp.exists(perf_csv_path):
    global_auc_tr, acc_tr, dice_tr, dice_from_bin_tr,\
    spec_tr, sens_tr = pd.read_csv(perf_csv_path).values[0]
    print('-- Performance in DRIVE training set had been pre-computed')
else:
    csv_path = osp.join('data', train_dataset, 'train.csv')
    if train_dataset=='HRF': csv_path = osp.join('data', train_dataset, 'train_fullRes.csv')
    preds, bin_preds, gts = get_labels_preds(path_to_preds, csv_path = csv_path)
    global_auc_tr, acc_tr, dice_tr, dice_from_bin_tr,\
    spec_tr, sens_tr, _ = compute_performance(preds, bin_preds, gts, save_path=save_path,
                                                      opt_threshold=opt_thresh_vl, cut_off=cut_off, mode='train')
    perf_df_train = pd.DataFrame({'auc': global_auc_tr,
                                  'acc': acc_tr,
                                  'dice/F1': dice_tr,
                                  'dice/F1_from_bin': dice_from_bin_tr,
                                  'spec': spec_tr,
                                  'sens': sens_tr}, index=[0])
    perf_df_train.to_csv(perf_csv_path, index=False)

In [ ]:
print('* Analyzing performance in ' + test_dataset + ' test set')
path_to_preds = osp.join(results_path, test_dataset, exp_name)
save_path = osp.join(path_to_preds, 'perf')
os.makedirs(save_path, exist_ok=True)
perf_csv_path = osp.join(save_path, 'test_performance.csv')

csv_name = 'test.csv' if train_dataset==test_dataset else 'test_all.csv'
if test_dataset=='HRF' and csv_name=='test_all.csv':
    print(8 * '-', ' For the entire HRF as a test set this can be very memory/time-consuming \
                     --> won\'t compute ROC curve//spec/sens, sorry', 8 * '-')
    no_auc=True
else: no_auc=False
path_test_csv = osp.join('data', test_dataset, csv_name)
preds, bin_preds, gts = get_labels_preds(path_to_preds, csv_path = path_test_csv)

In [ ]:
from skimport matplotlib.pyplot as plt

def imshow_pair(im, gdt):
    f, ax = plt.subplots(1, 2, figsize=(12,6))
    np_im = np.asarray(im)
    np_gdt = np.asarray(gdt)
    if len(np_im.shape) == 2:
        ax[0].imshow(np_im, cmap='gray'),  ax[0].axis('off')
    else:
        ax[0].imshow(np_im),  ax[0].axis('off')
    if len(np_gdt.shape) == 2:
        ax[1].imshow(np.asarray(gdt), cmap = 'gray'), ax[1].axis('off')
    else:
        ax[1].imshow(np.asarray(gdt)), ax[1].axis('off')
    plt.tight_layout()learn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(gts, preds)
plt.plot(fpr, tpr, label='ROC curve')

In [ ]:
from sklearn.metrics import auc
auc(fpr, tpr)

In [ ]:
from utils.evaluation import fast_auc as auc
auc(fpr, tpr)

In [ ]:
data_sets = ['DRIVE', 'CHASEDB', 'HRF', 'STARE']

In [ ]:
xxx = data_sets.remove('DRIVE')

In [ ]:
data_sets

In [ ]:
import os
import os.path as osp
import shutil
import pandas as pd
from PIL import Image
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

def imshow_pair(im, gdt):
    f, ax = plt.subplots(1, 2, figsize=(12,6))
    np_im = np.asarray(im)
    np_gdt = np.asarray(gdt)
    if len(np_im.shape) == 2:
        ax[0].imshow(np_im, cmap='gray'),  ax[0].axis('off')
    else:
        ax[0].imshow(np_im),  ax[0].axis('off')
    if len(np_gdt.shape) == 2:
        ax[1].imshow(np.asarray(gdt), cmap = 'gray'), ax[1].axis('off')
    else:
        ax[1].imshow(np.asarray(gdt)), ax[1].axis('off')
    plt.tight_layout()# process HRF data, generate CSVs
path_ims = 'data/HRF/images'
path_masks = 'data/HRF/mask'
path_gts = 'data/HRF/manual1'

path_ims_resized = 'data/HRF/images_resized'
os.makedirs(path_ims_resized, exist_ok=True)
path_masks_resized = 'data/HRF/mask_resized'
os.makedirs(path_masks_resized, exist_ok=True)
path_gts_resized = 'data/HRF/manual1_resized'
os.makedirs(path_gts_resized, exist_ok=True)

In [ ]:
all_im_names = sorted(os.listdir(path_ims))
all_mask_names = sorted(os.listdir(path_masks))
all_gt_names = sorted(os.listdir(path_gts))

# append paths
num_ims = len(all_im_names)
all_im_names = [osp.join(path_ims, n) for n in all_im_names]
all_mask_names = [osp.join(path_masks, n) for n in all_mask_names]
all_gt_names = [osp.join(path_gts, n) for n in all_gt_names]

df_hrf_all = pd.DataFrame({'im_paths': all_im_names,
                            'gt_paths': all_gt_names,
                            'mask_paths': all_mask_names})

In [ ]:
test_im_names = all_im_names[:num_ims//2]
train_im_names = all_im_names[num_ims//2:]

test_mask_names = all_mask_names[:num_ims//2]
train_mask_names = all_mask_names[num_ims//2:]

test_gt_names = all_gt_names[:num_ims//2]
train_gt_names = all_gt_names[num_ims//2:]

# use smaller images for trainining on HRF
train_im_names_resized = [n.replace(path_ims, path_ims_resized) for n in train_im_names]
train_mask_names_resized = [n.replace(path_masks, path_masks_resized) for n in train_mask_names]
train_gt_names_resized = [n.replace(path_gts, path_gts_resized) for n in train_gt_names]

In [ ]:
df_hrf_train_fullRes = pd.DataFrame({'im_paths': train_im_names,
                             'gt_paths': train_gt_names,
                             'mask_paths': train_mask_names})

df_hrf_test = pd.DataFrame({'im_paths': test_im_names,
                              'gt_paths': test_gt_names,
                               'mask_paths': test_mask_names})

In [ ]:
df_hrf_train = pd.DataFrame({'im_paths': train_im_names_resized,
                             'gt_paths': train_gt_names_resized,
                             'mask_paths': train_mask_names_resized})

In [ ]:
num_ims = len(df_hrf_train)
tr_ims = int(0.8*num_ims)

In [ ]:
df_hrf_train_fullRes, df_hrf_val_fullRes = df_hrf_train_fullRes[:tr_ims], df_hrf_train_fullRes[tr_ims:]
df_hrf_train, df_hrf_val = df_hrf_train[:tr_ims], df_hrf_train[tr_ims:]

In [ ]:
df_hrf_train.shape, df_hrf_val.shape

In [ ]:
df_hrf_train.to_csv('data/HRF/train.csv', index=False)
df_hrf_val.to_csv('data/HRF/val.csv', index=False)
df_hrf_test.to_csv('data/HRF/test.csv', index=False)
df_hrf_all.to_csv('data/HRF/test_all.csv', index=False)

df_hrf_train_fullRes.to_csv('data/HRF/train_fullRes.csv', index=False)
df_hrf_val_fullRes.to_csv('data/HRF/val_fullRes.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

def imshow_pair(im, gdt):
    f, ax = plt.subplots(1, 2, figsize=(12,6))
    np_im = np.asarray(im)
    np_gdt = np.asarray(gdt)
    if len(np_im.shape) == 2:
        ax[0].imshow(np_im, cmap='gray'),  ax[0].axis('off')
    else:
        ax[0].imshow(np_im),  ax[0].axis('off')
    if len(np_gdt.shape) == 2:
        ax[1].imshow(np.asarray(gdt), cmap = 'gray'), ax[1].axis('off')
    else:
        ax[1].imshow(np.asarray(gdt)), ax[1].axis('off')
    plt.tight_layout()

In [ ]:
from skimage.io import imread, imsave
from skimage.transform import resize
from skimage import img_as_ubyte

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
print('Resizing HRF images (**only** for training)\n')

In [ ]:
gt_name = train_gt_names[i]
gt = imread(gt_name)
gt_res = resize(gt, (gt.shape[0]//4, gt.shape[1]//4), order=0)
print(len(np.unique(gt)), len(np.unique(gt_res)))

In [ ]:
gt_name = train_gt_names[i]
gt = Image.open(gt_name)
gt_res = resize(gt, size=(gt.size[1]//4, gt.size[0]//4), interpolation=Image.NEAREST)
gt_res.save('what.png')
print(len(np.unique(gt)), len(np.unique(gt_res)))

In [ ]:
from torchvision.transforms.functional import resize

for i in tqdm(range(len(train_im_names))):
    im_name = train_im_names[i]
    im_name_out = train_im_names_resized[i]
    im = Image.open(im_name)
    im_res = resize(im, size=(im.size[1]//4, im.size[0]//4), interpolation=Image.NEAREST)
    im_res.save(im_name_out)

    mask_name = train_mask_names[i]
    mask_name_out = train_mask_names_resized[i]
    mask = Image.open(mask_name)
    mask_res = resize(mask, size=(mask.size[1]//4, mask.size[0]//4), interpolation=Image.NEAREST)
    mask_res.save(mask_name_out)

    gt_name = train_gt_names[i]
    gt_name_out = train_gt_names_resized[i]
    gt = Image.open(gt_name)
    gt_res = resize(gt, size=(gt.size[1]//4, gt.size[0]//4), interpolation=Image.NEAREST)
    gt_res.save(gt_name_out)


In [ ]:
imshow_pair(gt, gt_res)

In [ ]:
path_ims = 'data/CHASEDB/images'
path_masks = 'data/CHASEDB/chase-masks'
path_gts = 'data/CHASEDB/manual'

In [ ]:
all_im_names = sorted(os.listdir(path_ims))
all_mask_names = sorted(os.listdir(path_masks))
all_gt_names = sorted(os.listdir(path_gts))

# append paths
all_im_names = [osp.join(path_ims, n) for n in all_im_names]
all_mask_names = [osp.join(path_masks, n) for n in all_mask_names]
all_gt_names = [osp.join(path_gts, n) for n in all_gt_names if '1st' in n]

num_ims = len(all_im_names)
test_im_names = all_im_names[:num_ims//2]
train_im_names = all_im_names[num_ims//2:]

test_mask_names = all_mask_names[:num_ims//2]
train_mask_names = all_mask_names[num_ims//2:]

test_gt_names = all_gt_names[:num_ims//2]
train_gt_names = all_gt_names[num_ims//2:]

df_chasedb_all = pd.DataFrame({'im_paths': all_im_names,
                             'gt_paths': all_gt_names,
                             'mask_paths': all_mask_names})

In [ ]:
df_chasedb_train = pd.DataFrame({'im_paths': train_im_names,
                              'gt_paths': train_gt_names,
                              'mask_paths': train_mask_names})

df_chasedb_test = pd.DataFrame({'im_paths': test_im_names,
                              'gt_paths': test_gt_names,
                              'mask_paths': test_mask_names})

In [ ]:
num_ims = len(df_chasedb_train)
tr_ims = int(0.8*num_ims)
df_chasedb_train, df_chasedb_val = df_chasedb_train[:tr_ims], df_chasedb_train[tr_ims:]

In [ ]:
df_chasedb_train.to_csv('data/CHASEDB/train.csv', index=False)
df_chasedb_val.to_csv('data/CHASEDB/val.csv', index=False)
df_chasedb_test.to_csv('data/CHASEDB/test.csv', index=False)
df_chasedb_all.to_csv('data/CHASEDB/test_all.csv', index=False)

In [ ]:
path_ims = 'data/STARE/stare-images'
path_masks = 'data/STARE/stare-masks'
path_gts = 'data/STARE/labels-ah'

In [ ]:
all_im_names = sorted(os.listdir(path_ims))
all_mask_names = sorted(os.listdir(path_masks))
all_gt_names = sorted(os.listdir(path_gts))

# append paths
all_im_names = [osp.join(path_ims, n) for n in all_im_names]
all_mask_names = [osp.join(path_masks, n) for n in all_mask_names]
all_gt_names = [osp.join(path_gts, n) for n in all_gt_names]

num_ims = len(all_im_names)
test_im_names = all_im_names[:num_ims//2]
train_im_names = all_im_names[num_ims//2:]

test_mask_names = all_mask_names[:num_ims//2]
train_mask_names = all_mask_names[num_ims//2:]

test_gt_names = all_gt_names[:num_ims//2]
train_gt_names = all_gt_names[num_ims//2:]

df_stare_all = pd.DataFrame({'im_paths': all_im_names,
                             'gt_paths': all_gt_names,
                             'mask_paths': all_mask_names})

In [ ]:
df_stare_train = pd.DataFrame({'im_paths': train_im_names,
                              'gt_paths': train_gt_names,
                              'mask_paths': train_mask_names})

df_stare_test = pd.DataFrame({'im_paths': test_im_names,
                              'gt_paths': test_gt_names,
                              'mask_paths': test_mask_names})

In [ ]:
num_ims = len(df_stare_train)
tr_ims = int(0.8*num_ims)
df_stare_train, df_stare_val = df_stare_train[:tr_ims], df_stare_train[tr_ims:]

In [ ]:
df_stare_train.to_csv('data/STARE/train.csv', index=False)
df_stare_val.to_csv('data/STARE/val.csv', index=False)
df_stare_test.to_csv('data/STARE/test.csv', index=False)
df_stare_all.to_csv('data/STARE/test_all.csv', index=False)

In [ ]:
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from utils import paired_transforms_tv04 as p_tr

import os.path as osp
import pandas as pd
from PIL import Image
import numpy as np
from skimage.measure import regionprops


class TrainDataset(Dataset):
    def __init__(self, csv_path, transforms=None, label_values=None):
        df = pd.read_csv(csv_path)
        self.im_list = df.im_paths
        self.gt_list = df.gt_paths
        self.mask_list = df.mask_paths
        self.transforms = transforms
        self.label_values = label_values  # for use in label_encoding

    def label_encoding(self, gdt):
        gdt_gray = np.array(gdt.convert('L'))
        classes = np.arange(len(self.label_values))
        for i in classes:
            gdt_gray[gdt_gray == self.label_values[i]] = classes[i]
        return Image.fromarray(gdt_gray)

    def crop_to_fov(self, img, target, mask):
        minr, minc, maxr, maxc = regionprops(np.array(mask))[0].bbox
        im_crop = Image.fromarray(np.array(img)[minr:maxr, minc:maxc])
        tg_crop = Image.fromarray(np.array(target)[minr:maxr, minc:maxc])
        mask_crop = Image.fromarray(np.array(mask)[minr:maxr, minc:maxc])
        return im_crop, tg_crop, mask_crop

    def __getitem__(self, index):
        # load image and labels
        img = Image.open(self.im_list[index])
        target = Image.open(self.gt_list[index])
        mask = Image.open(self.mask_list[index]).convert('L')

        img, target, mask = self.crop_to_fov(img, target, mask)
        return target
        target = np.array(self.label_encoding(target))
        plt.figure(figsize=(12,12))
        plt.imshow(target, cmap='gray')
        plt.show()
        target[np.array(mask) == 0] = 0
        target = Image.fromarray(target)

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.im_list)

In [ ]:
!pwd

In [ ]:
path_data = '../little_unet/data/HRF'

In [ ]:
path_train_csv = osp.join(path_data, 'train.csv')
path_val_csv = osp.join(path_data, 'val.csv')

train_dataset = TrainDataset(csv_path=path_train_csv, label_values=[0, 255])
val_dataset = TrainDataset(csv_path=path_val_csv, label_values=[0, 255])

In [ ]:
im1 = Image.open('../little_unet/data/HRF/manual1_resized/08_g.tif').convert('L')
np.unique(np.array(im1))

In [ ]:
from skimage.io import imread, imsave
from skimage import img_as_ubyte

In [ ]:
# im1 = img_as_ubyte(imread('data/HRF/manual1_resized/08_g.tif').astype(bool))
im1 = imread('../little_unet/data/HRF/manual1/08_g.tif')
np.unique(np.array(im1))

In [ ]:
im2=train_dataset[0]

In [ ]:
imshow_pair(im1,im2)

In [ ]:
def label_encoding(gdt, label_values=[0, 255]):
    gdt_gray = np.array(gdt.convert('L'))
    classes = np.arange(len(label_values))
    for i in classes:
        gdt_gray[gdt_gray == label_values[i]] = classes[i]
    return Image.fromarray(gdt_gray)

In [ ]:
im3 = label_encoding(im2)
np.unique(np.array(im1))

In [ ]:
imshow_pair(im2, im3)